**1/14/21**

The purpose of this notebook is to analyze bacterial proteins identified by searching the Hybrid_Sample-Matched databases. First I'll just note all of the proteins identified across all the samples, then I'll analyze the attached GO numbers to see if any functional annotations are differentially more or less abundant in BV+ samples.

In [1]:
from elliot_utils import *
from scipy import stats

In [2]:
analysisPath = Path.cwd().joinpath('analysis_files/bacterial_analysis/')

In [ ]:
results = getOrderedFiles(HYBRID_RESULTS, '.tsv')
dbs = getOrderedFiles(HYBRID_DB, '.fasta')
refs = [ProtRef(x, disallowRepeats=False) for x in dbs]
bPeps = getFilteredPeptides(results, 'bacteria')

In [4]:
# Gather all the bacterial proteins, collapsing them by sequence, and count how many spectra they are associated with across all samples.
bacterialProteinHits = {} #key=amino acid sequence, value=ProteinHit object
proteinTotals = [0 for x in range(len(SAMPLE_NAMES))] #total number of bacterial spectra in each sample

In [5]:
for i in range(len(results)):
    res = results[i]
    ref = refs[i]
    bv = BV_STATUS[i]
    with res.open(mode='r') as infile:
        reader = csv.reader(infile, delimiter='\t')
        for row in reader:
            protType = determineIDType(row)
            if protType == 'first':
                continue
            if not isSignificant(row):
                break
            if row[PEPTIDE] in bPeps and protType == 'bacteria':
                proteinTotals[i] += 1
                hits = getProteinHitList(row, 'bacteria')
                hitSequences = set()
                for hit in hits:
                    protein = ref.getProt(hit)
                    if not protein.sequence in bacterialProteinHits.keys():
                        bacterialProteinHits[protein.sequence] = ProteinHit(protein.copy())
                    else:
                        for t in protein.taxa:
                            bacterialProteinHits[protein.sequence].protein.addTaxa(t)
                    hitSequences.add(protein.sequence)
                for seq in hitSequences:
                    bacterialProteinHits[seq].incrementCount(bv)

In [6]:
summaryPath = analysisPath.joinpath('protein_summary_bacteria_v2.csv')

In [7]:
with summaryPath.open(mode='w', newline='') as outfile:
    rows = [['ID', 'Protein', 'Taxa', 'BV- Count', 'BV+ Count', 'Total Count']]
    for p in bacterialProteinHits.values():
        rows.append([p.id, p.protein.name, p.protein.getTaxaString(), str(p.getCount('-')), str(p.getCount('+')), str(p.getCount())])
    writer = csv.writer(outfile)
    for row in rows:
        writer.writerow(row)

In [7]:
# Perform the same analysis as above, but only count each spectrum once
strictBacteriaHits = {} #key=protID, value=protein hit object
for i in range(len(results)):
    res = results[i]
    ref = refs[i]
    bv = BV_STATUS[i]
    with res.open(mode='r') as infile:
        reader = csv.reader(infile, delimiter='\t')
        for row in reader:
            protType = determineIDType(row)
            if protType == 'first':
                continue
            if not isSignificant(row):
                break
            if row[PEPTIDE] in bPeps and protType == 'bacteria':
                proteinTotals[i] += 1
                hits = getProteinHitList(row, 'bacteria')
                hitTaxa = set()
                toIncrement = None
                for hit in hits:
                    prot = ref.getProt(hit)
                    for t in prot.taxa:
                        hitTaxa.add(t)
                    if hit in strictBacteriaHits.keys() and toIncrement == None:
                        toIncrement = hit
                if toIncrement == None:
                    toIncrement = hits[0]
                    strictBacteriaHits[toIncrement] = ProteinHit(ref.getProt(toIncrement))
                strictBacteriaHits[toIncrement].incrementCount(bv)
                for t in hitTaxa:
                    strictBacteriaHits[toIncrement].addTaxa(t)

In [21]:
strictSummaryPath = analysisPath.joinpath('protein_summary_bacteria_strict.csv')

In [18]:
with strictSummaryPath.open(mode='w', newline='') as outfile:
    rows = [['ID', 'Protein', 'Taxa', 'BV- Count', 'BV+ Count', 'Total Count']]
    for p in strictBacteriaHits.values():
        rows.append([p.id, p.protein.name, p.getTaxaString(), str(p.getCount('-')), str(p.getCount('+')), str(p.getCount())])
    writer = csv.writer(outfile)
    for row in rows:
        writer.writerow(row)

I'm going to analyze the bacterial proteins by the annotations given to them by eggnog-mapper. It's a little tricky to decide which annotation to use for analysis, as none of the EC number, GO number, KEGG number, etc, classifications were applied evenly to all the proteins I would be interested in. Therefore, I'm going to use the "Preferred_name" column to group proteins. However, that still isn't perfect, as some proteins that I'm interested in such as lactate dehydrogenase can have different annotations (i.e. 'ldh' and 'ldhA'). So, I'm going to hand-build a dictionary associating those annotations I'm interested in with a function, (i.e. 'ldh' and 'ldhA' both ='Lactic Acid Dehydrogenase'), and analyze those hand-picked functions.

In [9]:
name2func = {} #key="Preferred_name", value=function

In [10]:
#key="Preferred_name", value=function
name2func = {
    # Fermentation
    'ldh':'Lactic acid dehydrogenase',
    'ldhA':'Lactic acid dehydrogenase',
    'ackA':'Acetate kinase',
    'pflB':'Pyruvate formate lyase',
    'adhA':'Alcohol dehydrogenase',
    'adhE':'Alcohol dehydrogenase',
    'adhP':'Alcohol dehydrogenase',
    'fucO':'Alcohol dehydrogenase',
    'adh1':'Alcohol dehydrogenase',
    'mdh':'Malate dehydrogenase',
    'sdhA':'Succinate dehydrogenase',
    'frdB':'Succinate dehydrogenase',
    'sucD':'Succinate-semialdehyde dehydrogenase',
    # Glycolysis
    'eno':'Enolase',
    'fba':'Fructose-bisphosphate aldolase',
    'pgk':'Phosphoglycerate kinase',
    'gap':'Glyceraldehyde-3-phosphate dehydrogenase',
    'pyk':'Pyruvate kinase',
    'pfkA':'6-phosphofructokinase',
    'tkt':'Transketolase',
    'gpmA':'Phosphoglycerate mutase',
    'gpmB':'Phosphoglycerate mutase',
    # Gluconeogenesis
    'pckA':'Phosphoenolpyruvate carboxykinase',
    'ppsA':'Pyruvate, phosphate dikinase',
    'tpiA':'Triosephosphate isomerase',
    # Carbohydrate metabolism
    'pulA':'Extracellular alpha-amylase',
    'amyE':'Extracellular alpha-amylase',
    'amyB7':'Extracellular alpha-amylase',
    'tvaII':'Intracellular alpha-amylase',
    'malQ':'4-alpha-glucanotransferase',
    'glgP':'Glycogen phosphorylase',
    'glgB':'Glycogen branching enzyme',
    # Growth processes
    'ctc':'Ribosome',
    'efp':'Ribosome',
    'frr':'Ribosome',
    'hpf':'Ribosome',
    'fusA':'Ribosome',
    'fusA2':'Ribosome',
    'infA':'Ribosome',
    'infB':'Ribosome',
    'infC':'Ribosome',
    'tuf':'Ribosome',
    'tsf':'Ribosome',
    'rpsA':'Ribosome',
    'rpsB':'Ribosome',
    'rpsC':'Ribosome',
    'rpsD':'Ribosome',
    'rpsE':'Ribosome',
    'rpsF':'Ribosome',
    'rpsG':'Ribosome',
    'rpsH':'Ribosome',
    'rpsI':'Ribosome',
    'rpsJ':'Ribosome',
    'rpsK':'Ribosome',
    'rpsL':'Ribosome',
    'rpsM':'Ribosome',
    'rpsN':'Ribosome',
    'rpsO':'Ribosome',
    'rpsP':'Ribosome',
    'rpsQ':'Ribosome',
    'rpsR':'Ribosome',
    'rpsS':'Ribosome',
    'rpsT':'Ribosome',
    'rplA':'Ribosome',
    'rplB':'Ribosome',
    'rplD':'Ribosome',
    'rplE':'Ribosome',
    'rplF':'Ribosome',
    'rplI':'Ribosome',
    'rplJ':'Ribosome',
    'rplK':'Ribosome',
    'rplL':'Ribosome',
    'rplM':'Ribosome',
    'rplN':'Ribosome',
    'rplO':'Ribosome',
    'rplP':'Ribosome',
    'rplQ':'Ribosome',
    'rplR':'Ribosome',
    'rplS':'Ribosome',
    'rplT':'Ribosome',
    'rplU':'Ribosome',
    'rplV':'Ribosome',
    'rplW':'Ribosome',
    'rplX':'Ribosome',
    'rpmA':'Ribosome',
    'rpmB':'Ribosome',
    'rpmC':'Ribosome',
    'rpmD':'Ribosome',
    'ychF':'Ribosome',
    'ftsZ':'Cell division',
    'sepF':'Cell division',
    'spoVG':'Cell division',
    'atpA':'ATP synthase',
    'atpD':'ATP synthase',
    'atpF':'ATP synthase',
    'rpoA':'RNA Polymerase',
    'rpoB':'RNA Polymerase',
    'rpoC':'RNA Polymerase',
    'glnA':'Glutamine synthase',
    'traM':'Conjugation',
    'tcsA':'ABC transporter',
    'ugpC':'ABC transporter',
    'dppA':'ABC transporter',
    'msmK':'ABC transporter',
    'msmX':'ABC transporter',
    'oppD':'ABC transporter',
    'pstS':'ABC transporter',
    'oppA':'ABC transporter',
    'potA':'ABC transporter',
    'bcrA':'ABC transporter',
    # Phage
    'casC':'CRISPR',
    # Peptide catabolism
    'pepC2':'Intracellular Peptidase',
    'pepD':'Intracellular Peptidase',
    'pepD_2':'Intracellular Peptidase',
    'pepN':'Intracellular Peptidase',
    'htrA':'Extracellular Peptidase',
    'clpP':'Extracellular Peptidase',
    'pepO':'Extracellular Peptidase',
    'gluA':'Glutamate transporter',
    'gluD':'Glutamate dehydrogenase',
    'gdh':'Glutamate dehydrogenase',
    'ftcD':'Glutamate formimidoyltransferase',
    'argJ':'Glutamate <-> Ornithine',
    'grdB':'Glycine reductase',
    'hutH':'Histidine ammonia-lyase',
    'arcA':'Arginine deiminase',
    'argE':'Acetylornithine deacetylase',
    'argF':'Ornithine transcarbamoylase',
    'tyrA':'Prephenate dehydrogenase',
    'aspA':'Aspartate ammonia-lyase',
    'asnA':'Aspartate ammonia-lyase',
    'amt':'Ammonium transporter',
    # Fatty acid catabolixm
    'bcd2':'Fatty acid catabolism',
    'citF':'Citrate lyase',
    # Motility
    'flaA':'Flagellum',
    'fliC':'Flagellum',
    'hag':'Flagellum',
    # Metals
    'ftnA':'Iron storage',
    'feoB':'Iron importer',
    'cbiK':'Cobalt chelatase',
    'yfeA':'Zinc uptake',
    'copB':'Copper-exporting ATPase',
    # Oxidant/Antioxidant
    'ahpC':'Peroxiredoxin',
    'rbo':'Superoxide reductase'
}

In [12]:
class FuncInfo:
    def __init__(self, functionName):
        self.function = functionName
        self.taxa = set()
        self.counts = [0 for x in range(len(SAMPLE_NAMES))]
    
    def addTaxa(self, newTaxa):
        self.taxa.add(newTaxa)
    
    def getTaxaString(self):
        taxaString = ''
        for t in self.taxa:
            taxaString += f'{t};'
        return taxaString
    
    def getAbundances(self, totalCounts, transformed):
        toReturn = []
        for i in range(len(totalCounts)):
            abundance = math.log2(1 + (self.counts[i] / totalCounts[i])) if transformed else self.counts[i] / totalCounts[i]
            toReturn.append(abundance)
        return toReturn

In [13]:
funcPath = Path.cwd().joinpath('analysis_files/functional_analysis/')
annotationFile = funcPath.joinpath('eggnog_annotations_bacteria.tsv')

In [17]:
prot2func = {} #key=protID, value=function
annotData = pd.read_csv(annotationFile, delimiter='\t')
for index, row in annotData.iterrows():
    protID = row['query']
    annot = row['Preferred_name']
    if annot in name2func.keys():
        prot2func[protID] = name2func[annot]

In [13]:
funcHolder = {} #key=function, value=FuncInfo object
for func in name2func.values():
    funcHolder[func] = FuncInfo(func)

In [14]:
totals = [0 for x in range(len(SAMPLE_NAMES))]
for i in range(len(results)):
    res = results[i]
    ref = refs[i]
    with res.open(mode='r') as infile:
        reader = csv.reader(infile, delimiter='\t')
        for row in reader:
            protType = determineIDType(row)
            if protType == 'first':
                continue
            if not isSignificant(row):
                break
            if row[PEPTIDE] in bPeps and protType == 'bacteria':
                totals[i] += 1
                hits = getProteinHitList(row, 'bacteria')
                hitFuncs = set()
                hitTaxa = set()
                for prot in hits:
                    if prot in prot2func.keys():
                        hitFuncs.add(prot2func[prot])
                        for t in ref.getProt(prot).taxa:
                            hitTaxa.add(t)
                for func in hitFuncs:
                    funcHolder[func].counts[i] += 1
                    for t in hitTaxa:
                        funcHolder[func].addTaxa(t)

In [15]:
# Returns two separate lists, the first for BV- itesm, the second for BV+
def separateByBV(itemList):
    bvNeg = []
    bvPos = []
    for i in range(len(BV_STATUS)):
        if BV_STATUS[i] == '-':
            bvNeg.append(itemList[i])
        else:
            bvPos.append(itemList[i])
    return (bvNeg, bvPos)

# Returns the mean of the array
def mean(numList):
    return sum(numList) / len(numList)

In [17]:
toWrite = [['Function', 'Taxa', 'p-value', 'BV- Avg Abundance', 'BV- Avg Transformed Abundance', 'BV- StDev', 'BV+ Avg Abundance', 'BV+ Avg Transformed Abundance', 'BV+ StDev']]
for f in funcHolder.values():
    bvNegAbund, bvPosAbund = separateByBV(f.getAbundances(totals, False))
    bvNegTransformed, bvPosTransformed = separateByBV(f.getAbundances(totals, True))
    res = stats.mannwhitneyu(bvNegTransformed, bvPosTransformed)
    pVal = res.pvalue
    toWrite.append([f.function, f.getTaxaString(), str(pVal), str(mean(bvNegAbund)), str(mean(bvNegTransformed)), str(stats.tstd(bvNegTransformed)), str(mean(bvPosAbund)), str(mean(bvPosTransformed)), str(stats.tstd(bvPosTransformed))])

In [18]:
funcOutPath = analysisPath.joinpath('function_differences.csv')

In [19]:
with funcOutPath.open(mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in toWrite:
        writer.writerow(row)

What taxa make extracellular peptidases?

In [19]:
peptidaseIDs = []
for protID, function in prot2func.items():
    if function == 'Extracellular Peptidase':
        peptidaseIDs.append(protID)

In [22]:
strictHitsDF = pd.read_csv(strictSummaryPath)

In [23]:
bacterialProteaseCounts = {} # key=taxa string, value=[bv- count, bv+ count]
for index, row in strictHitsDF.iterrows():
    if row['ID'] in peptidaseIDs:
        if not row['Taxa'] in bacterialProteaseCounts:
            bacterialProteaseCounts[row['Taxa']] = [0, 0]
        bacterialProteaseCounts[row['Taxa']][0] += row['BV- Count']
        bacterialProteaseCounts[row['Taxa']][1] += row['BV+ Count']

In [26]:
proteaseSummaryPath = analysisPath.joinpath('bacterial_strict_proteases.csv')
with proteaseSummaryPath.open(mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['Taxa', 'BV- Count', 'BV+ Count'])
    for taxaString, counts in bacterialProteaseCounts.items():
        writer.writerow([taxaString, str(counts[0]), str(counts[1])])